In [1]:
import cv2
import support_utility_openvino
import numpy as np

In [2]:
model_path = "/home/prateek/prateek_space/intel_model_dump/intel/single-image-super-resolution-1033/FP32/single-image-super-resolution-1033.xml"
cpu_extension = "../build_samples/intel64/Release/lib/libcpu_extension.so"
device = "CPU"

In [3]:
model_detection = support_utility_openvino.async_infer()

model_detection.load_model(model_path,device,cpu_extension)

model inputs : {'0': <openvino.inference_engine.ie_api.DataPtr object at 0x7f9d20ef1830>, '1': <openvino.inference_engine.ie_api.DataPtr object at 0x7f9d1851ef30>}
model outputs :  {'90': <openvino.inference_engine.ie_api.DataPtr object at 0x7f9d20ef1830>}
loading sample element


In [16]:
channel = "/media/prateek/PK_HDD/Movies/Hindi/Ramayan - The Legend of Prince Ram(1992)-8plH4CW7Feo.mp4"
# channel = "/media/prateek/shared_space/Notebooks/abg/openvino/Notebooks/output_vids/distance_measurement/stacker_bay_hot_mill/comb_out.mp4"
cap = cv2.VideoCapture(channel)
cap.set(1,10000)
while True:
    ret,frame = cap.read()
    bicupic_orig = cv2.resize(frame,(1920,1080), interpolation = cv2.INTER_CUBIC) 
    model_detection.preprocess_frame(frame)
    bicupic =  bicupic_orig.transpose((2, 0, 1))
#     model_detection.exec_net.requests[0].inputs['0'][:] = model_detection.in_frame
#     model_detection.exec_net.requests[0].inputs['1'][:] = np.expand_dims(bicupic, axis=0)
    model_detection.exec_net.requests[0].infer({'0': model_detection.in_frame,'1':np.expand_dims(bicupic, axis=0)})
    res = [model_detection.exec_net.requests[0].outputs[node] for node in model_detection.out_blob ][0][0]
    res *= 255
    # for ensurance that all values in necessary diapasone
    res = np.clip(res, 0., 255.)
    # transpose CHW -> HWC and convert to unsigned int
    res = res.transpose((1, 2, 0)).astype(np.uint8)
    cv2.imshow("preview",res)
    cv2.imshow("preview1",bicupic_orig)
    k = cv2.waitKey(1)
    if k==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [8]:
model_detection.input_blob

'0'